## Pandas pivot##

With the pivot function you can transpose some columns of a dataframe, taking the name of the new columns from another field.

In [1]:
import pandas as pd
import numpy as np

In [2]:
# Example data
df = pd.DataFrame({"id": ['7c883c5b','7c883c5b','e7dcce49','e7dcce49'],
                   "item": [1,2,1,3], "part":[1,1,1,1],"score": [0.00,0.50,0.50,0.00], "max_score": [1.00,0.50,1.00,1.00], "percentage": [0,100,50,0]})
df

,id,item,part,score,max_score,percentage
0,7c883c5b,1,1,0.0,1.0,0
1,7c883c5b,2,1,0.5,0.5,100
2,e7dcce49,1,1,0.5,1.0,50
3,e7dcce49,3,1,0.0,1.0,0


In [3]:
# column field to name the columns.
df["column"] =  "i" + df["item"].astype(str) + "p" + df["part"].astype(str)
df

,id,item,part,score,max_score,percentage,column
0,7c883c5b,1,1,0.0,1.0,0,i1p1
1,7c883c5b,2,1,0.5,0.5,100,i2p1
2,e7dcce49,1,1,0.5,1.0,50,i1p1
3,e7dcce49,3,1,0.0,1.0,0,i3p1


In [4]:
# pivot transposes "score", "max_score", "percentage" according to "column", generating a multiindex as column names.
# Same values of "column" go to the same column. Missing values get NaN.
df1 = df.pivot(index="id", columns="column", values=["score", "max_score", "percentage"])
df1

score           max_score           percentage            
column    i1p1 i2p1 i3p1      i1p1 i2p1 i3p1       i1p1   i2p1 i3p1
id                                                                 
7c883c5b   0.0  0.5  NaN       1.0  0.5  NaN        0.0  100.0  NaN
e7dcce49   0.5  NaN  0.0       1.0  NaN  1.0       50.0    NaN  0.0

In [5]:
# We need to join the names in the multiindex levels in the inverse order, so the level are swaped
df2 = df1.swaplevel(axis=1)
df2

column,i1p1,i2p1,i3p1,i1p1,i2p1,i3p1,i1p1,i2p1,i3p1
,score,score,score,max_score,max_score,max_score,percentage,percentage,percentage
id,,,,,,,,,
7c883c5b,0.0,0.5,NaN,1.0,0.5,NaN,0.0,100.0,NaN
e7dcce49,0.5,NaN,0.0,1.0,NaN,1.0,50.0,NaN,0.0


In [6]:
# The levels are joined and become a normal index (not a multiindex anymore)
df2.set_axis(df2.columns.map(' '.join), axis=1, inplace=True)
df2

,i1p1 score,i2p1 score,i3p1 score,i1p1 max_score,i2p1 max_score,i3p1 max_score,i1p1 percentage,i2p1 percentage,i3p1 percentage
id,,,,,,,,,
7c883c5b,0.0,0.5,NaN,1.0,0.5,NaN,0.0,100.0,NaN
e7dcce49,0.5,NaN,0.0,1.0,NaN,1.0,50.0,NaN,0.0


In [7]:
# Reorder columns
df3 = df2[["i1p1 score", "i1p1 max_score", "i1p1 percentage", "i2p1 score", "i2p1 max_score", "i2p1 percentage", "i3p1 score", "i3p1 max_score", "i3p1 percentage"]]
df3

,i1p1 score,i1p1 max_score,i1p1 percentage,i2p1 score,i2p1 max_score,i2p1 percentage,i3p1 score,i3p1 max_score,i3p1 percentage
id,,,,,,,,,
7c883c5b,0.0,1.0,0.0,0.5,0.5,100.0,NaN,NaN,NaN
e7dcce49,0.5,1.0,50.0,NaN,NaN,NaN,0.0,1.0,0.0


## Group by ##
groupby function split a dataframe generating tuples of (value of the groupby field, dataframe with the group).

In [8]:
df = pd.DataFrame(
    [
        ("bird", "Falconiformes", 389.0),
        ("bird", "Psittaciformes", 24.0),
        ("mammal", "Carnivora", 80.2),
        ("mammal", "Primates", np.nan),
        ("mammal", "Carnivora", 58),
    ],
    index=["falcon", "parrot", "lion", "monkey", "leopard"],
    columns=("class", "order", "max_speed"),
)
df

,class,order,max_speed
falcon,bird,Falconiformes,389.0
parrot,bird,Psittaciformes,24.0
lion,mammal,Carnivora,80.2
monkey,mammal,Primates,NaN
leopard,mammal,Carnivora,58.0


In [9]:
grouped = df.groupby("class", axis=0)
for g in grouped:
    print("Group class:", g[0])
    print(g[1], "\n")

Group class: bird
       class           order  max_speed
falcon  bird   Falconiformes      389.0
parrot  bird  Psittaciformes       24.0 

Group class: mammal
          class      order  max_speed
lion     mammal  Carnivora       80.2
monkey   mammal   Primates        NaN
leopard  mammal  Carnivora       58.0 

